In [9]:
# import libraries related to querying links and downloading files from the web
import importlib
import pandas as pd
from pipmag import file_utils as fu
from pipmag import selector_utils as su
from pipmag import ads_utils as ads
# If you get the error ModuleNotFoundError: No module named 'pipmag', run the following line and restart the kernel:
# %pip install -e ..

In [4]:
# get the latest pickle file
latest_updated_la_palma_obs_data_file = fu.get_latest_file('../data/la_palma_obs_data_*.pkl')

Latest file: ../data/la_palma_obs_data_20230706_081731.pkl


In [5]:
df = pd.read_pickle(latest_updated_la_palma_obs_data_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666 entries, 0 to 665
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date_time    666 non-null    datetime64[ns]
 1   year         666 non-null    int64         
 2   month        666 non-null    int64         
 3   day          666 non-null    int64         
 4   time         666 non-null    object        
 5   instruments  666 non-null    object        
 6   target       0 non-null      object        
 7   comments     0 non-null      object        
 8   video_links  666 non-null    object        
 9   image_links  666 non-null    object        
 10  links        666 non-null    object        
 11  num_links    666 non-null    int64         
 12  polarimetry  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 72.8+ KB


In [ ]:
# create a widget to display movies based on year, month, day and time
# and to update the target, instrumnets and comments columns of the dataframe
selector = su.VideoSelector2(df, ['target', 'instruments', 'polarimetry', 'comments'])
selector.create_widget()

In [10]:
# 🔍 ADS Search
index = 36
search = ads.ADS_Search(df)
search.get_results(index, pretty_print=True)

In [12]:
# save the updated dataframe as a pickle file
df.to_pickle('../data/'+ fu.add_timestamp('la_palma_obs_data.pkl'))